<a href="https://colab.research.google.com/github/Jundula/Neural-networks/blob/main/ARISOY_ABD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Step 1: Load Stock Data
stock_data_path = "data.csv"  # File uploaded to Colab
stock_data = pd.read_csv(stock_data_path)

# Convert date column to datetime format
stock_data['date'] = pd.to_datetime(stock_data['date'])

# Calculate market equity (ME)
stock_data['ME'] = stock_data['PRC'].abs() * stock_data['SHROUT']

# Calculate natural log of ME
stock_data['ln_ME'] = np.log(stock_data['ME'])

# Sort by PERMNO and date
stock_data = stock_data.sort_values(by=['PERMNO', 'date'])

# Step 2: Load Fama-French Factors
ff_factors_path = "F-F_Research_Data_Factors.csv"  # File uploaded to Colab

# Skip the first 3 rows (descriptive text) and use row 4 as the header
ff_factors = pd.read_csv(ff_factors_path, skiprows=3)

# Assign a label to the first column (date column)
ff_factors.columns = ['date'] + list(ff_factors.columns[1:])

# Drop rows where the 'date' column is NaN
ff_factors = ff_factors.dropna(subset=['date'])

# Filter out rows where the 'date' column does not match the YYYYMM format
ff_factors = ff_factors[ff_factors['date'].str.isdigit() & (ff_factors['date'].str.len() == 6)]

# Convert date column to datetime format
ff_factors['date'] = pd.to_datetime(ff_factors['date'], format='%Y%m')

# Ensure the dates align with the stock data
ff_factors = ff_factors.set_index('date').resample('M').last().reset_index()

# Step 3: Load q-Factor Data
q_factors_path = "q5_factors_monthly_2023.csv"  # File uploaded to Colab
q_factors = pd.read_csv(q_factors_path)

# Combine year and month into a single date column
q_factors['date'] = pd.to_datetime(q_factors['year'].astype(str) + '-' + q_factors['month'].astype(str) + '-01')

# Rename columns for consistency
q_factors.rename(columns={'R_MKT': 'mkt_excess_return', 'R_ME': 'ME'}, inplace=True)

# Drop unnecessary columns (year and month), but keep 'date', 'mkt_excess_return', and 'ME'
q_factors = q_factors[['date', 'mkt_excess_return', 'ME']]

# Ensure the dates align with the stock data
q_factors = q_factors.set_index('date').resample('M').last().reset_index()

q_factors

<ipython-input-2-7d21e3bbdbbd>:40: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ff_factors = ff_factors.set_index('date').resample('M').last().reset_index()
<ipython-input-2-7d21e3bbdbbd>:56: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  q_factors = q_factors.set_index('date').resample('M').last().reset_index()


,date,mkt_excess_return,ME
0,1967-01-31,8.1852,6.8122
1,1967-02-28,0.7557,1.6235
2,1967-03-31,4.0169,1.9836
3,1967-04-30,3.8786,-0.6700
4,1967-05-31,-4.2807,2.7366
...,...,...,...
679,2023-08-31,-2.3886,-2.3035
680,2023-09-30,-5.2404,-0.1586
681,2023-10-31,-3.1794,-2.6272
682,2023-11-30,8.8694,-0.7873


In [3]:
# Step 4: Merge Stock Data with Factors
merged_data = pd.merge(stock_data, ff_factors, on='date', how='inner')
merged_data = pd.merge(merged_data, q_factors, on='date', how='inner')

# Ensure RET and RF columns are numeric
merged_data['RET'] = pd.to_numeric(merged_data['RET'], errors='coerce')
merged_data['RF'] = pd.to_numeric(merged_data['RF'], errors='coerce')

# Calculate excess returns for each stock
merged_data['excess_return'] = merged_data['RET'] - merged_data['RF']

In [4]:
merged_data

,PERMNO,date,SHRCD,EXCHCD,PRC,RET,SHROUT,ME_x,ln_ME,Mkt-RF,SMB,HML,RF,mkt_excess_return,ME_y,excess_return
0,10001,1986-09-30,11,3,6.3750,-0.003077,991,6317.6250,8.751099,-8.60,2.36,3.22,0.45,-8.6052,0.6344,-0.453077
1,10001,1986-10-31,11,3,6.6250,0.039216,991,6565.3750,8.789565,4.66,-2.50,-1.42,0.46,4.6836,-1.7298,-0.420784
2,10001,1986-12-31,11,3,7.0000,0.015000,991,6937.0000,8.844625,-3.27,0.13,0.36,0.49,-3.2308,0.8393,-0.475000
3,10001,1987-03-31,11,3,6.3750,0.036800,991,6317.6250,8.751099,1.64,0.45,1.61,0.47,1.6042,-0.6612,-0.433200
4,10001,1987-06-30,11,3,5.8750,0.051429,991,5822.1250,8.669421,3.94,-2.12,1.04,0.48,3.8854,-1.5507,-0.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739911,55466,1998-04-30,11,1,33.1250,-0.053571,10498,347746.2500,12.759228,0.73,0.07,0.94,0.43,0.7319,0.4968,-0.483571
739912,55466,1998-06-30,11,1,28.1250,-0.048626,10499,295284.3750,12.595694,3.18,-3.21,-1.96,0.41,3.1842,-3.2841,-0.458626
739913,55466,1998-07-31,11,1,25.0000,-0.111111,10499,262475.0000,12.477911,-2.46,-4.91,-1.78,0.40,-2.4070,-5.7045,-0.511111
739914,55466,1998-08-31,11,1,22.0000,-0.117500,10525,231550.0000,12.352551,-16.08,-5.69,3.53,0.43,-16.0240,-6.1863,-0.547500


In [5]:
# Step 5: Rolling Regression Function
def rolling_regression(data, window=60):
    betas_mkt = []
    betas_me = []

    for i in range(window, len(data)):
        # Extract the rolling window
        window_data = data.iloc[i-window:i]

        # Drop rows with NaN values in any of the required columns
        window_data = window_data.dropna(subset=['mkt_excess_return', 'ln_ME', 'excess_return'])

        # Skip if there are fewer than 2 observations (minimum required for regression)
        if len(window_data) < 2:
            betas_mkt.append(np.nan)
            betas_me.append(np.nan)
            continue

        # Define independent variables (MKT and ln_ME) and dependent variable (excess return)
        X = window_data[['mkt_excess_return', 'ln_ME']]
        y = window_data['excess_return']

        # Fit the regression model
        model = LinearRegression().fit(X, y)

        # Save the betas
        betas_mkt.append(model.coef_[0])  # Beta for MKT
        betas_me.append(model.coef_[1])   # Beta for ln_ME

    # Return betas starting from the 61st month
    return pd.DataFrame({
        'date': data['date'][window:],
        'beta_mkt': betas_mkt,
        'beta_me': betas_me
    })

In [6]:

# Step 6: Apply Rolling Regression to Each Stock
grouped = merged_data.groupby('PERMNO')
results = grouped.apply(lambda x: rolling_regression(x))

# Reset index to flatten the results
results = results.reset_index(level=0).rename(columns={'level_0': 'PERMNO'})

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-0b375a60f9f5>", line 3, in <cell line: 0>
    results = grouped.apply(lambda x: rolling_regression(x))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/groupby/groupby.py", line 1824, in apply
    result = self._python_apply_general(f, self._selected_obj)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/groupby/groupby.py", line 1885, in _python_apply_general
    values, mutated = self._grouper.apply_groupwise(f, data, self.axis)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/groupby/ops.py", line 919, in apply_groupwise
    res = f(group)
   

TypeError: object of type 'NoneType' has no len()

## **Non-GPU Accelerated Version**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Step 1: Load Stock Data
stock_data_path = "data.csv"  # File uploaded to Colab
stock_data = pd.read_csv(stock_data_path)

# Convert date column to datetime format
stock_data['date'] = pd.to_datetime(stock_data['date'])

# Calculate market equity (ME)
stock_data['ME'] = stock_data['PRC'].abs() * stock_data['SHROUT']

# Calculate natural log of ME
stock_data['ln_ME'] = np.log(stock_data['ME'])

# Sort by PERMNO and date
stock_data = stock_data.sort_values(by=['PERMNO', 'date'])

# Step 2: Load Fama-French Factors
ff_factors_path = "F-F_Research_Data_Factors.csv"  # File uploaded to Colab

# Skip the first 3 rows (descriptive text) and use row 4 as the header
ff_factors = pd.read_csv(ff_factors_path, skiprows=3)

# Assign a label to the first column (date column)
ff_factors.columns = ['date'] + list(ff_factors.columns[1:])

# Drop rows where the 'date' column is NaN
ff_factors = ff_factors.dropna(subset=['date'])

# Filter out rows where the 'date' column does not match the YYYYMM format
ff_factors = ff_factors[ff_factors['date'].str.isdigit() & (ff_factors['date'].str.len() == 6)]

# Convert date column to datetime format
ff_factors['date'] = pd.to_datetime(ff_factors['date'], format='%Y%m')

# Ensure the dates align with the stock data
ff_factors = ff_factors.set_index('date').resample('M').last().reset_index()

# Step 3: Load q-Factor Data
q_factors_path = "q5_factors_monthly_2023.csv"  # File uploaded to Colab
q_factors = pd.read_csv(q_factors_path)

# Combine year and month into a single date column
q_factors['date'] = pd.to_datetime(q_factors['year'].astype(str) + '-' + q_factors['month'].astype(str) + '-01')

# Rename columns for consistency
q_factors.rename(columns={'R_MKT': 'mkt_excess_return', 'R_ME': 'ME'}, inplace=True)

# Drop unnecessary columns (year and month), but keep 'date', 'mkt_excess_return', and 'ME'
q_factors = q_factors[['date', 'mkt_excess_return', 'ME']]

# Ensure the dates align with the stock data
q_factors = q_factors.set_index('date').resample('M').last().reset_index()

# Step 4: Merge Stock Data with Factors
merged_data = pd.merge(stock_data, ff_factors, on='date', how='inner')
merged_data = pd.merge(merged_data, q_factors, on='date', how='inner')

# Ensure RET and RF columns are numeric
merged_data['RET'] = pd.to_numeric(merged_data['RET'], errors='coerce')
merged_data['RF'] = pd.to_numeric(merged_data['RF'], errors='coerce')

# Calculate excess returns for each stock
merged_data['excess_return'] = merged_data['RET'] - merged_data['RF']

# Step 5: Rolling Regression Function
def rolling_regression(data, window=60):
    betas_mkt = []
    betas_me = []

    for i in range(window, len(data)):
        # Extract the rolling window
        window_data = data.iloc[i-window:i]

        # Drop rows with NaN values in any of the required columns
        window_data = window_data.dropna(subset=['mkt_excess_return', 'ln_ME', 'excess_return'])

        # Skip if there are fewer than 2 observations (minimum required for regression)
        if len(window_data) < 2:
            betas_mkt.append(np.nan)
            betas_me.append(np.nan)
            continue

        # Define independent variables (MKT and ln_ME) and dependent variable (excess return)
        X = window_data[['mkt_excess_return', 'ln_ME']]
        y = window_data['excess_return']

        # Fit the regression model
        model = LinearRegression().fit(X, y)

        # Save the betas
        betas_mkt.append(model.coef_[0])  # Beta for MKT
        betas_me.append(model.coef_[1])   # Beta for ln_ME

    # Return betas starting from the 61st month
    return pd.DataFrame({
        'date': data['date'][window:],
        'beta_mkt': betas_mkt,
        'beta_me': betas_me
    })
# Step 6: Apply Rolling Regression to Each Stock
grouped = merged_data.groupby('PERMNO')
results = grouped.apply(lambda x: rolling_regression(x))

# Reset index to flatten the results
results = results.reset_index(level=0).rename(columns={'level_0': 'PERMNO'})

# Save the results to a CSV file
betas_results_path = "betas_results.csv"
results.to_csv(betas_results_path, index=False)

# Step 7: Portfolio Formation (Monthly Deciles)
def form_portfolios(data, beta_column, weight_column=None):
    """
    Forms 10 decile portfolios based on beta values.
    If weight_column is None, portfolios are equally-weighted.
    Otherwise, portfolios are value-weighted using the weight_column.
    """
    # Rank stocks into deciles based on beta values
    data['decile'] = pd.qcut(data[beta_column], q=10, labels=False)

    # Group by decile and calculate portfolio returns
    if weight_column:
        # Value-weighted returns
        portfolio_returns = data.groupby('decile').apply(
            lambda x: np.average(x['RET'], weights=x[weight_column])
        )
    else:
        # Equally-weighted returns
        portfolio_returns = data.groupby('decile')['RET'].mean()

    return portfolio_returns

# Step 8: Monthly Portfolio Returns
# Load betas results
betas_results = pd.read_csv(betas_results_path)

# Merge betas with stock data
portfolio_data = pd.merge(betas_results, merged_data, on=['PERMNO', 'date'], how='inner')

# Form portfolios monthly
monthly_portfolios = portfolio_data.groupby('date').apply(
    lambda x: form_portfolios(x, beta_column='beta_me', weight_column='ME_x')
)

# Save portfolio returns to a CSV file
monthly_portfolios_path = "monthly_portfolio_returns.csv"
monthly_portfolios.to_csv(monthly_portfolios_path)

# Step 9: Arbitrage Portfolio (Long Portfolio 10, Short Portfolio 1)
arbitrage_portfolio = monthly_portfolios[10] - monthly_portfolios[1]

# Save arbitrage portfolio returns to a CSV file
arbitrage_portfolio_path = "arbitrage_portfolio_returns.csv"
arbitrage_portfolio.to_csv(arbitrage_portfolio_path)

# Step 10: Performance Metrics
def calculate_metrics(returns):
    avg_return = returns.mean()
    std_return = returns.std()
    sharpe_ratio = avg_return / std_return
    t_stat = avg_return / (std_return / np.sqrt(len(returns)))
    return pd.Series({
        'Average Return': avg_return,
        'Standard Deviation': std_return,
        'Sharpe Ratio': sharpe_ratio,
        't-statistic': t_stat
    })

# Calculate metrics for arbitrage portfolio
metrics = calculate_metrics(arbitrage_portfolio)
print(metrics)

# Save metrics to a CSV file
metrics_path = "arbitrage_portfolio_metrics.csv"
metrics.to_csv(metrics_path)

## **GPU Accelerated Version**

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Step 1: Load Stock Data
stock_data_path = "data.csv"  # File uploaded to Colab
stock_data = pd.read_csv(stock_data_path)

# Convert date column to datetime format
stock_data['date'] = pd.to_datetime(stock_data['date'])

# Filter data: Prices > $1, Exchange codes 1, 2, or 3, Share codes 10 or 11
filtered_data = stock_data[
    (stock_data['PRC'].abs() > 1) &
    (stock_data['EXCHCD'].isin([1, 2, 3])) &
    (stock_data['SHRCD'].isin([10, 11]))
]

# Calculate market equity (ME)
filtered_data['ME'] = filtered_data['PRC'].abs() * filtered_data['SHROUT']

# Calculate natural log of ME
filtered_data['ln_ME'] = np.log(filtered_data['ME'])

# Sort by PERMNO and date
filtered_data = filtered_data.sort_values(by=['PERMNO', 'date'])

# Step 2: Load Fama-French Factors
ff_factors_path = "F-F_Research_Data_Factors.csv"  # File uploaded to Colab

# Skip the first 3 rows (descriptive text) and use row 4 as the header
ff_factors = pd.read_csv(ff_factors_path, skiprows=3)

# Assign a label to the first column (date column)
ff_factors.columns = ['date'] + list(ff_factors.columns[1:])

# Drop rows where the 'date' column is NaN
ff_factors = ff_factors.dropna(subset=['date'])

# Filter out rows where the 'date' column does not match the YYYYMM format
ff_factors = ff_factors[ff_factors['date'].str.isdigit() & (ff_factors['date'].str.len() == 6)]

# Convert date column to datetime format
ff_factors['date'] = pd.to_datetime(ff_factors['date'], format='%Y%m')

# Rename columns for consistency
ff_factors.rename(columns={'Mkt-RF': 'mkt_excess_return'}, inplace=True)

# Ensure the dates align with the stock data
ff_factors = ff_factors.set_index('date').resample('M').last().reset_index()

# Step 3: Load q-Factor Data
q_factors_path = "q5_factors_monthly_2023.csv"  # File uploaded to Colab
q_factors = pd.read_csv(q_factors_path)

# Combine year and month into a single date column
q_factors['date'] = pd.to_datetime(q_factors['year'].astype(str) + '-' + q_factors['month'].astype(str) + '-01')

# Rename columns for consistency
q_factors.rename(columns={'R_MKT': 'mkt_excess_return', 'R_ME': 'ME'}, inplace=True)

# Drop unnecessary columns (year and month), but keep 'date', 'mkt_excess_return', and 'ME'
q_factors = q_factors[['date', 'mkt_excess_return', 'ME']]

# Ensure the dates align with the stock data
q_factors = q_factors.set_index('date').resample('M').last().reset_index()

# Step 4: Merge Stock Data with Factors
merged_data = pd.merge(filtered_data, ff_factors, on='date', how='inner')
merged_data = pd.merge(merged_data, q_factors, on='date', how='inner')

# Debug: Print columns after merging
print("Merged Data Columns:", merged_data.columns)

# Ensure RET and RF columns are numeric
merged_data['RET'] = pd.to_numeric(merged_data['RET'], errors='coerce')
merged_data['RF'] = pd.to_numeric(merged_data['RF'], errors='coerce')

# Calculate excess returns for each stock
merged_data['excess_return'] = merged_data['RET'] - merged_data['RF']

# Step 5: Rolling Regression Function (Using TensorFlow for GPU Acceleration)
def rolling_regression_tensorflow(data, window=60):
    betas_mkt = []
    betas_me = []

    for i in range(window, len(data)):
        # Extract the rolling window
        window_data = data.iloc[i-window:i]

        # Drop rows with NaN values in any of the required columns
        window_data = window_data.dropna(subset=['mkt_excess_return', 'ln_ME', 'excess_return'])

        # Skip if there are fewer than 2 observations (minimum required for regression)
        if len(window_data) < 2:
            betas_mkt.append(np.nan)
            betas_me.append(np.nan)
            continue

        # Define independent variables (MKT and ln_ME) and dependent variable (excess return)
        X = window_data[['mkt_excess_return', 'ln_ME']].values
        y = window_data['excess_return'].values

        # Convert to TensorFlow tensors and move to GPU
        X_tf = tf.convert_to_tensor(X, dtype=tf.float32)
        y_tf = tf.convert_to_tensor(y, dtype=tf.float32)

        # Add a column of ones for the intercept term
        X_tf = tf.concat([tf.ones((X_tf.shape[0], 1), dtype=tf.float32), X_tf], axis=1)

        # Compute coefficients using the normal equation: β = (X^T X)^(-1) X^T y
        beta = tf.linalg.inv(tf.transpose(X_tf) @ X_tf) @ tf.transpose(X_tf) @ y_tf

        # Save the betas (skip the intercept term)
        betas_mkt.append(beta.numpy()[1])  # Beta for MKT
        betas_me.append(beta.numpy()[2])   # Beta for ln_ME

    # Return betas starting from the 61st month
    return pd.DataFrame({
        'date': data['date'][window:],
        'beta_mkt': betas_mkt,
        'beta_me': betas_me
    })



<ipython-input-10-fc482b188d34>:50: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ff_factors = ff_factors.set_index('date').resample('M').last().reset_index()
<ipython-input-10-fc482b188d34>:66: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  q_factors = q_factors.set_index('date').resample('M').last().reset_index()


Merged Data Columns: Index(['PERMNO', 'date', 'SHRCD', 'EXCHCD', 'PRC', 'RET', 'SHROUT', 'ME_x',
       'ln_ME', 'mkt_excess_return_x', 'SMB', 'HML', 'RF',
       'mkt_excess_return_y', 'ME_y'],
      dtype='object')


In [11]:
merged_data

,PERMNO,date,SHRCD,EXCHCD,PRC,RET,SHROUT,ME_x,ln_ME,mkt_excess_return_x,SMB,HML,RF,mkt_excess_return_y,ME_y,excess_return
0,10001,1986-09-30,11,3,6.3750,-0.003077,991,6317.6250,8.751099,-8.60,2.36,3.22,0.45,-8.6052,0.6344,-0.453077
1,10001,1986-10-31,11,3,6.6250,0.039216,991,6565.3750,8.789565,4.66,-2.50,-1.42,0.46,4.6836,-1.7298,-0.420784
2,10001,1986-12-31,11,3,7.0000,0.015000,991,6937.0000,8.844625,-3.27,0.13,0.36,0.49,-3.2308,0.8393,-0.475000
3,10001,1987-03-31,11,3,6.3750,0.036800,991,6317.6250,8.751099,1.64,0.45,1.61,0.47,1.6042,-0.6612,-0.433200
4,10001,1987-06-30,11,3,5.8750,0.051429,991,5822.1250,8.669421,3.94,-2.12,1.04,0.48,3.8854,-1.5507,-0.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739911,55466,1998-04-30,11,1,33.1250,-0.053571,10498,347746.2500,12.759228,0.73,0.07,0.94,0.43,0.7319,0.4968,-0.483571
739912,55466,1998-06-30,11,1,28.1250,-0.048626,10499,295284.3750,12.595694,3.18,-3.21,-1.96,0.41,3.1842,-3.2841,-0.458626
739913,55466,1998-07-31,11,1,25.0000,-0.111111,10499,262475.0000,12.477911,-2.46,-4.91,-1.78,0.40,-2.4070,-5.7045,-0.511111
739914,55466,1998-08-31,11,1,22.0000,-0.117500,10525,231550.0000,12.352551,-16.08,-5.69,3.53,0.43,-16.0240,-6.1863,-0.547500


In [12]:
# Step 6: Apply Rolling Regression to Each Stock
grouped = merged_data.groupby('PERMNO')

In [16]:
results = grouped.apply(lambda x: rolling_regression_tensorflow(x))

# Reset index to flatten the results
results = results.reset_index(level=0).rename(columns={'level_0': 'PERMNO'})

# Save the results to a CSV file
betas_results_path = "betas_results.csv"
results.to_csv(betas_results_path, index=False)

# Step 7: Portfolio Formation (Monthly Deciles)
def form_portfolios(data, beta_column, weight_column=None):
    """
    Forms 10 decile portfolios based on beta values.
    If weight_column is None, portfolios are equally-weighted.
    Otherwise, portfolios are value-weighted using the weight_column.
    """
    # Rank stocks into deciles based on beta values
    data['decile'] = pd.qcut(data[beta_column], q=10, labels=False)

    # Group by decile and calculate portfolio returns
    if weight_column:
        # Value-weighted returns
        portfolio_returns = data.groupby('decile').apply(
            lambda x: np.average(x['RET'], weights=x[weight_column])
        )
    else:
        # Equally-weighted returns
        portfolio_returns = data.groupby('decile')['RET'].mean()

    return portfolio_returns

# Step 8: Monthly Portfolio Returns
# Load betas results
betas_results = pd.read_csv(betas_results_path)

# Merge betas with stock data
portfolio_data = pd.merge(betas_results, merged_data, on=['PERMNO', 'date'], how='inner')

# Form portfolios monthly
monthly_portfolios = portfolio_data.groupby('date').apply(
    lambda x: form_portfolios(x, beta_column='beta_me', weight_column='ME')
)

# Save portfolio returns to a CSV file
monthly_portfolios_path = "monthly_portfolio_returns.csv"
monthly_portfolios.to_csv(monthly_portfolios_path)

# Step 9: Arbitrage Portfolio (Long Portfolio 10, Short Portfolio 1)
arbitrage_portfolio = monthly_portfolios[10] - monthly_portfolios[1]

# Save arbitrage portfolio returns to a CSV file
arbitrage_portfolio_path = "arbitrage_portfolio_returns.csv"
arbitrage_portfolio.to_csv(arbitrage_portfolio_path)

# Step 10: Performance Metrics
def calculate_metrics(returns):
    avg_return = returns.mean()
    std_return = returns.std()
    sharpe_ratio = avg_return / std_return
    t_stat = avg_return / (std_return / np.sqrt(len(returns)))
    return pd.Series({
        'Average Return': avg_return,
        'Standard Deviation': std_return,
        'Sharpe Ratio': sharpe_ratio,
        't-statistic': t_stat
    })

# Calculate metrics for arbitrage portfolio
metrics = calculate_metrics(arbitrage_portfolio)
print(metrics)

# Save metrics to a CSV file
metrics_path = "arbitrage_portfolio_metrics.csv"
metrics.to_csv(metrics_path)

KeyError: ['mkt_excess_return']

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Step 1: Load Stock Data
stock_data_path = "data.csv"  # File uploaded to Colab
stock_data = pd.read_csv(stock_data_path)

# Convert date column to datetime format
stock_data['date'] = pd.to_datetime(stock_data['date'])

# Filter data: Prices > $1, Exchange codes 1, 2, or 3, Share codes 10 or 11
filtered_data = stock_data[
    (stock_data['PRC'].abs() > 1) &
    (stock_data['EXCHCD'].isin([1, 2, 3])) &
    (stock_data['SHRCD'].isin([10, 11]))
]

# Calculate market equity (ME)
filtered_data['ME'] = filtered_data['PRC'].abs() * filtered_data['SHROUT']

# Calculate natural log of ME
filtered_data['ln_ME'] = np.log(filtered_data['ME'])

# Sort by PERMNO and date
filtered_data = filtered_data.sort_values(by=['PERMNO', 'date'])

# Step 2: Load Fama-French Factors
ff_factors_path = "F-F_Research_Data_Factors.csv"  # File uploaded to Colab

# Skip the first 3 rows (descriptive text) and use row 4 as the header
ff_factors = pd.read_csv(ff_factors_path, skiprows=3)

# Assign a label to the first column (date column)
ff_factors.columns = ['date'] + list(ff_factors.columns[1:])

# Drop rows where the 'date' column is NaN
ff_factors = ff_factors.dropna(subset=['date'])

# Filter out rows where the 'date' column does not match the YYYYMM format
ff_factors = ff_factors[ff_factors['date'].str.isdigit() & (ff_factors['date'].str.len() == 6)]

# Convert date column to datetime format
ff_factors['date'] = pd.to_datetime(ff_factors['date'], format='%Y%m')

# Rename columns for consistency
ff_factors.rename(columns={'Mkt-RF': 'mkt_excess_return'}, inplace=True)

# Ensure the dates align with the stock data
ff_factors = ff_factors.set_index('date').resample('M').last().reset_index()

# Step 3: Load q-Factor Data (Replace with your assigned factor)
q_factors_path = "q5_factors_monthly_2023.csv"  # File uploaded to Colab
q_factors = pd.read_csv(q_factors_path)

# Combine year and month into a single date column
q_factors['date'] = pd.to_datetime(q_factors['year'].astype(str) + '-' + q_factors['month'].astype(str) + '-01')

# Rename columns for consistency (Replace 'R_ME' with your assigned factor)
q_factors.rename(columns={'R_MKT': 'mkt_excess_return', 'R_ME': 'allocated_factor'}, inplace=True)

# Drop unnecessary columns (year and month), but keep 'date', 'mkt_excess_return', and 'allocated_factor'
q_factors = q_factors[['date', 'mkt_excess_return', 'allocated_factor']]

# Ensure the dates align with the stock data
q_factors = q_factors.set_index('date').resample('M').last().reset_index()

# Step 4: Merge Stock Data with Factors
merged_data = pd.merge(filtered_data, ff_factors, on='date', how='inner')
merged_data = pd.merge(merged_data, q_factors, on='date', how='inner')

# Resolve duplicate columns if necessary
if 'mkt_excess_return_x' in merged_data.columns and 'mkt_excess_return_y' in merged_data.columns:
    merged_data['mkt_excess_return'] = merged_data['mkt_excess_return_x']
    merged_data.drop(columns=['mkt_excess_return_x', 'mkt_excess_return_y'], inplace=True)

# Debug: Print columns after merging
print("Merged Data Columns:", merged_data.columns)

# Ensure RET and RF columns are numeric
merged_data['RET'] = pd.to_numeric(merged_data['RET'], errors='coerce')
merged_data['RF'] = pd.to_numeric(merged_data['RF'], errors='coerce')

# Calculate excess returns for each stock
merged_data['excess_return'] = merged_data['RET'] - merged_data['RF']

# Step 5: Rolling Regression Function (Using TensorFlow for GPU Acceleration)
def rolling_regression_tensorflow(data, window=60):
    betas_mkt = []
    betas_allocated = []

    for i in range(window, len(data)):
        # Extract the rolling window
        window_data = data.iloc[i-window:i]

        # Drop rows with NaN values in any of the required columns
        window_data = window_data.dropna(subset=['mkt_excess_return', 'allocated_factor', 'excess_return'])

        # Skip if there are fewer than 2 observations (minimum required for regression)
        if len(window_data) < 2:
            betas_mkt.append(np.nan)
            betas_allocated.append(np.nan)
            continue

        # Define independent variables (MKT and Allocated Factor) and dependent variable (excess return)
        X = window_data[['mkt_excess_return', 'allocated_factor']].values
        y = window_data['excess_return'].values

        try:
            # Convert to TensorFlow tensors and move to GPU
            X_tf = tf.convert_to_tensor(X, dtype=tf.float32)
            y_tf = tf.convert_to_tensor(y, dtype=tf.float32)

            # Add a column of ones for the intercept term
            X_tf = tf.concat([tf.ones((X_tf.shape[0], 1), dtype=tf.float32), X_tf], axis=1)

            # Compute coefficients using the normal equation: β = (X^T X)^(-1) X^T y
            beta = tf.linalg.inv(tf.transpose(X_tf) @ X_tf) @ tf.transpose(X_tf) @ y_tf

            # Save the betas (skip the intercept term)
            betas_mkt.append(beta.numpy()[1])  # Beta for MKT
            betas_allocated.append(beta.numpy()[2])  # Beta for Allocated Factor
        except Exception as e:
            print(f"Error in regression at index {i}: {e}")
            betas_mkt.append(np.nan)
            betas_allocated.append(np.nan)

    # Return betas starting from the 61st month
    return pd.DataFrame({
        'date': data['date'][window:],
        'beta_mkt': betas_mkt,
        'beta_allocated': betas_allocated
    })

# Step 6: Apply Rolling Regression to Each Stock
grouped = merged_data.groupby('PERMNO')
results = grouped.apply(lambda x: rolling_regression_tensorflow(x))

# Reset index to flatten the results
results = results.reset_index(level=0).rename(columns={'level_0': 'PERMNO'})

# Save the results to a CSV file
betas_results_path = "betas_results.csv"
results.to_csv(betas_results_path, index=False)

# Step 7: Portfolio Formation (Monthly Deciles)
def form_portfolios(data, beta_column, weight_column=None):
    """
    Forms 10 decile portfolios based on beta values.
    If weight_column is None, portfolios are equally-weighted.
    Otherwise, portfolios are value-weighted using the weight_column.
    """
    # Check if there are enough unique beta values for decile ranking
    if len(data[beta_column].dropna().unique()) < 10:
        return pd.Series({i: np.nan for i in range(10)})

    # Rank stocks into deciles based on beta values
    try:
        data['decile'] = pd.qcut(data[beta_column], q=10, labels=False)
    except ValueError:
        return pd.Series({i: np.nan for i in range(10)})

    # Group by decile and calculate portfolio returns
    if weight_column:
        # Value-weighted returns
        portfolio_returns = data.groupby('decile').apply(
            lambda x: np.average(x['RET'], weights=x[weight_column])
        )
    else:
        # Equally-weighted returns
        portfolio_returns = data.groupby('decile')['RET'].mean()

    return portfolio_returns

# Step 8: Monthly Portfolio Returns
# Load betas results
betas_results = pd.read_csv(betas_results_path)

# Merge betas with stock data
portfolio_data = pd.merge(betas_results, merged_data, on=['PERMNO', 'date'], how='inner')

# Form portfolios monthly
monthly_portfolios = portfolio_data.groupby('date').apply(
    lambda x: form_portfolios(x, beta_column='beta_allocated', weight_column='ME')
)

# Save portfolio returns to a CSV file
monthly_portfolios_path = "monthly_portfolio_returns.csv"
monthly_portfolios.to_csv(monthly_portfolios_path)

# Step 9: Arbitrage Portfolio (Long Portfolio 10, Short Portfolio 1)
arbitrage_portfolio = monthly_portfolios.get(10, pd.Series(index=monthly_portfolios.index, data=np.nan)) - \
                      monthly_portfolios.get(1, pd.Series(index=monthly_portfolios.index, data=np.nan))

# Save arbitrage portfolio returns to a CSV file
arbitrage_portfolio_path = "arbitrage_portfolio_returns.csv"
arbitrage_portfolio.to_csv(arbitrage_portfolio_path)

# Step 10: Performance Metrics
def calculate_metrics(returns):
    avg_return = returns.mean()
    std_return = returns.std()
    sharpe_ratio = avg_return / std_return
    t_stat = avg_return / (std_return / np.sqrt(len(returns)))
    return pd.Series({
        'Average Return': avg_return,
        'Standard Deviation': std_return,
        'Sharpe Ratio': sharpe_ratio,
        't-statistic': t_stat
    })

# Calculate metrics for arbitrage portfolio
if arbitrage_portfolio.notna().sum() > 0:
    metrics = calculate_metrics(arbitrage_portfolio)
    print(metrics)
else:
    print("Arbitrage portfolio contains no valid returns.")
    metrics = pd.Series({
        'Average Return': np.nan,
        'Standard Deviation': np.nan,
        'Sharpe Ratio': np.nan,
        't-statistic': np.nan
    })

# Save metrics to a CSV file
metrics_path = "arbitrage_portfolio_metrics.csv"
metrics.to_csv(metrics_path)

<ipython-input-17-ce1fbcda17cb>:50: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ff_factors = ff_factors.set_index('date').resample('M').last().reset_index()
<ipython-input-17-ce1fbcda17cb>:66: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  q_factors = q_factors.set_index('date').resample('M').last().reset_index()


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Error in regression at index 109: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:GPU:0}} In[0] and In[1] has different ndims: [3,60] vs. [60] [Op:MatMul] name: 
Error in regression at index 110: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:GPU:0}} In[0] and In[1] has different ndims: [3,60] vs. [60] [Op:MatMul] name: 
Error in regression at index 111: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:GPU:0}} In[0] and In[1] has different ndims: [3,60] vs. [60] [Op:MatMul] name: 
Error in regression at index 112: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:GPU:0}} In[0] and In[1] has different ndims: [3,60] vs. [60] [Op:MatMul] name: 
Error in regression at index 113: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:GPU:0}} In[0] and In[1] has diff

KeyboardInterrupt: 

In [28]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Step 1: Load Stock Data
stock_data_path = "data.csv"  # File uploaded to Colab
stock_data = pd.read_csv(stock_data_path)

# Convert date column to datetime format
stock_data['date'] = pd.to_datetime(stock_data['date'])

# Filter data: Prices > $1, Exchange codes 1, 2, or 3, Share codes 10 or 11
filtered_data = stock_data[
    (stock_data['PRC'].abs() > 1) &
    (stock_data['EXCHCD'].isin([1, 2, 3])) &
    (stock_data['SHRCD'].isin([10, 11]))
]

# Calculate market equity (ME)
filtered_data['ME'] = filtered_data['PRC'].abs() * filtered_data['SHROUT']

# Calculate natural log of ME
filtered_data['ln_ME'] = np.log(filtered_data['ME'])

# Sort by PERMNO and date
filtered_data = filtered_data.sort_values(by=['PERMNO', 'date'])

# Step 2: Load Fama-French Factors
ff_factors_path = "F-F_Research_Data_Factors.csv"  # File uploaded to Colab

# Skip the first 3 rows (descriptive text) and use row 4 as the header
ff_factors = pd.read_csv(ff_factors_path, skiprows=3)

# Assign a label to the first column (date column)
ff_factors.columns = ['date'] + list(ff_factors.columns[1:])

# Drop rows where the 'date' column is NaN
ff_factors = ff_factors.dropna(subset=['date'])

# Filter out rows where the 'date' column does not match the YYYYMM format
ff_factors = ff_factors[ff_factors['date'].str.isdigit() & (ff_factors['date'].str.len() == 6)]

# Convert date column to datetime format
ff_factors['date'] = pd.to_datetime(ff_factors['date'], format='%Y%m')

# Rename columns for consistency
ff_factors.rename(columns={'Mkt-RF': 'mkt_excess_return'}, inplace=True)

# Ensure the dates align with the stock data
ff_factors = ff_factors.set_index('date').resample('M').last().reset_index()

# Step 3: Load q-Factor Data (Replace with your assigned factor)
q_factors_path = "q5_factors_monthly_2023.csv"  # File uploaded to Colab
q_factors = pd.read_csv(q_factors_path)

# Combine year and month into a single date column
q_factors['date'] = pd.to_datetime(q_factors['year'].astype(str) + '-' + q_factors['month'].astype(str) + '-01')

# Rename columns for consistency (Replace 'R_ME' with your assigned factor)
q_factors.rename(columns={'R_MKT': 'mkt_excess_return', 'R_ME': 'allocated_factor'}, inplace=True)

# Drop unnecessary columns (year and month), but keep 'date', 'mkt_excess_return', and 'allocated_factor'
q_factors = q_factors[['date', 'mkt_excess_return', 'allocated_factor']]

# Ensure the dates align with the stock data
q_factors = q_factors.set_index('date').resample('M').last().reset_index()

# Step 4: Merge Stock Data with Factors
merged_data = pd.merge(filtered_data, ff_factors, on='date', how='inner')
merged_data = pd.merge(merged_data, q_factors, on='date', how='inner')

# Resolve duplicate columns if necessary
if 'mkt_excess_return_x' in merged_data.columns and 'mkt_excess_return_y' in merged_data.columns:
    merged_data['mkt_excess_return'] = merged_data['mkt_excess_return_x']
    merged_data.drop(columns=['mkt_excess_return_x', 'mkt_excess_return_y'], inplace=True)

# Debug: Print columns after merging
print("Merged Data Columns:", merged_data.columns)

# Ensure RET and RF columns are numeric
merged_data['RET'] = pd.to_numeric(merged_data['RET'], errors='coerce')
merged_data['RF'] = pd.to_numeric(merged_data['RF'], errors='coerce')

# Calculate excess returns for each stock
merged_data['excess_return'] = merged_data['RET'] - merged_data['RF']

# Step 5: Create a Smaller Subset for Testing
# Limit the date range to a shorter period (e.g., 1973–1983)
subset_data = merged_data[(merged_data['date'] >= '1973-01-01') & (merged_data['date'] <= '1983-12-31')]

# Randomly select a subset of PERMNOs (e.g., 100 stocks)
unique_permnos = subset_data['PERMNO'].unique()
sample_permnos = np.random.choice(unique_permnos, size=100, replace=False)  # Randomly select 100 PERMNOs
subset_data = subset_data[subset_data['PERMNO'].isin(sample_permnos)]

# Verify the subset
print(f"Subset Data Shape: {subset_data.shape}")
print(f"Unique PERMNOs in Subset: {len(subset_data['PERMNO'].unique())}")
# Step 6: Rolling Regression Function (Using TensorFlow for GPU Acceleration)

def rolling_regression_tensorflow(data, window=12):  # Reduced window size for testing
    betas_mkt = []
    betas_allocated = []

    for i in range(window, len(data)):
        # Extract the rolling window
        window_data = data.iloc[i-window:i]

        # Drop rows with NaN values in any of the required columns
        window_data = window_data.dropna(subset=['mkt_excess_return', 'allocated_factor', 'excess_return'])

        # Skip if there are fewer than 2 observations (minimum required for regression)
        if len(window_data) < 2:
            betas_mkt.append(float(beta.numpy()[1]))  # Ensure scalar value for MKT
            betas_allocated.append(float(beta.numpy()[2]))  # Ensure scalar value for Allocated Factor
            continue

        # Define independent variables (MKT and Allocated Factor) and dependent variable (excess return)
        X = window_data[['mkt_excess_return', 'allocated_factor']].values
        y = window_data['excess_return'].values

        try:
            # Debug: Print shapes of X and y
            print(f"Index {i}: X shape = {X.shape}, y shape = {y.shape}")

            # Convert to TensorFlow tensors and move to GPU
            X_tf = tf.convert_to_tensor(X, dtype=tf.float32)
            y_tf = tf.convert_to_tensor(y, dtype=tf.float32)

            # Reshape y_tf to be a 2D array with shape (n_samples, 1)
            y_tf = tf.reshape(y_tf, (-1, 1))

            # Add a column of ones for the intercept term
            X_tf = tf.concat([tf.ones((X_tf.shape[0], 1), dtype=tf.float32), X_tf], axis=1)

            # Compute coefficients using the normal equation: β = (X^T X)^(-1) X^T y
            beta = tf.linalg.inv(tf.transpose(X_tf) @ X_tf) @ tf.transpose(X_tf) @ y_tf

            # Save the betas (skip the intercept term)
            betas_mkt.append(beta.numpy()[1])  # Beta for MKT
            betas_allocated.append(beta.numpy()[2])  # Beta for Allocated Factor
        except Exception as e:
            print(f"Error in regression at index {i}: {e}")
            betas_mkt.append(np.nan)
            betas_allocated.append(np.nan)

    # Return betas starting from the 13th month (window + 1)
    return pd.DataFrame({
        'date': data['date'][window:],
        'beta_mkt': betas_mkt,
        'beta_allocated': betas_allocated
    })

# Step 7: Apply Rolling Regression to Each Stock in the Subset
grouped = subset_data.groupby('PERMNO')
results_subset = grouped.apply(lambda x: rolling_regression_tensorflow(x, window=12))  # Use reduced window

# Reset index to flatten the results
results_subset = results_subset.reset_index(level=0).rename(columns={'level_0': 'PERMNO'})

# Inspect the results
print("Subset Results Shape:", results_subset.shape)
print(results_subset.head())

# Step 8: Portfolio Formation (Monthly Deciles)
def form_portfolios(data, beta_column, weight_column=None):
    """
    Forms 10 decile portfolios based on beta values.
    If weight_column is None, portfolios are equally-weighted.
    Otherwise, portfolios are value-weighted using the weight_column.
    """
    # Ensure beta_column contains valid scalar values
    data[beta_column] = pd.to_numeric(data[beta_column], errors='coerce')

    # Check if there are enough unique beta values for decile ranking
    unique_betas = data[beta_column].dropna().unique()
    if len(unique_betas) < 10:
        print(f"Not enough unique beta values ({len(unique_betas)}) to form 10 deciles.")
        return pd.Series({i: np.nan for i in range(10)})

    # Rank stocks into deciles based on beta values
    try:
        data['decile'] = pd.qcut(data[beta_column], q=10, labels=False)
    except ValueError as e:
        print(f"Error in pd.qcut: {e}")
        return pd.Series({i: np.nan for i in range(10)})

    # Group by decile and calculate portfolio returns
    if weight_column:
        # Value-weighted returns
        portfolio_returns = data.groupby('decile').apply(
            lambda x: np.average(x['RET'], weights=x[weight_column])
        )
    else:
        # Equally-weighted returns
        portfolio_returns = data.groupby('decile')['RET'].mean()

    return portfolio_returns

# Step 9: Monthly Portfolio Returns
# Merge betas with stock data
portfolio_data_subset = pd.merge(results_subset, subset_data, on=['PERMNO', 'date'], how='inner')

# Form portfolios monthly
monthly_portfolios_subset = portfolio_data_subset.groupby('date').apply(
    lambda x: form_portfolios(x, beta_column='beta_allocated', weight_column='ME')
)

# Inspect the portfolio returns
print("Monthly Portfolios Subset:")
print(monthly_portfolios_subset.head())

# Step 10: Arbitrage Portfolio (Long Portfolio 10, Short Portfolio 1)
arbitrage_portfolio_subset = monthly_portfolios_subset.get(10, pd.Series(index=monthly_portfolios_subset.index, data=np.nan)) - \
                              monthly_portfolios_subset.get(1, pd.Series(index=monthly_portfolios_subset.index, data=np.nan))

# Save arbitrage portfolio returns to a CSV file
arbitrage_portfolio_path = "arbitrage_portfolio_returns_subset.csv"
arbitrage_portfolio_subset.to_csv(arbitrage_portfolio_path)

# Step 11: Performance Metrics
def calculate_metrics(returns):
    avg_return = returns.mean()
    std_return = returns.std()
    sharpe_ratio = avg_return / std_return
    t_stat = avg_return / (std_return / np.sqrt(len(returns)))
    return pd.Series({
        'Average Return': avg_return,
        'Standard Deviation': std_return,
        'Sharpe Ratio': sharpe_ratio,
        't-statistic': t_stat
    })

# Calculate metrics for arbitrage portfolio
if arbitrage_portfolio_subset.notna().sum() > 0:
    metrics_subset = calculate_metrics(arbitrage_portfolio_subset)
    print("Subset Metrics:")
    print(metrics_subset)
else:
    print("Arbitrage portfolio contains no valid returns.")
    metrics_subset = pd.Series({
        'Average Return': np.nan,
        'Standard Deviation': np.nan,
        'Sharpe Ratio': np.nan,
        't-statistic': np.nan
    })

# Save metrics to a CSV file
metrics_path = "arbitrage_portfolio_metrics_subset.csv"
metrics_subset.to_csv(metrics_path)

<ipython-input-28-7581b272b43a>:50: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ff_factors = ff_factors.set_index('date').resample('M').last().reset_index()
<ipython-input-28-7581b272b43a>:66: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  q_factors = q_factors.set_index('date').resample('M').last().reset_index()


Merged Data Columns: Index(['PERMNO', 'date', 'SHRCD', 'EXCHCD', 'PRC', 'RET', 'SHROUT', 'ME',
       'ln_ME', 'SMB', 'HML', 'RF', 'allocated_factor', 'mkt_excess_return'],
      dtype='object')
Subset Data Shape: (5313, 15)
Unique PERMNOs in Subset: 100
Index 12: X shape = (12, 2), y shape = (12,)
Index 13: X shape = (12, 2), y shape = (12,)
Index 14: X shape = (12, 2), y shape = (12,)
Index 15: X shape = (12, 2), y shape = (12,)
Index 16: X shape = (12, 2), y shape = (12,)
Index 17: X shape = (12, 2), y shape = (12,)
Index 18: X shape = (12, 2), y shape = (12,)
Index 19: X shape = (12, 2), y shape = (12,)
Index 20: X shape = (12, 2), y shape = (12,)
Index 21: X shape = (12, 2), y shape = (12,)
Index 22: X shape = (12, 2), y shape = (12,)
Index 23: X shape = (12, 2), y shape = (12,)
Index 24: X shape = (12, 2), y shape = (12,)
Index 25: X shape = (12, 2), y shape = (12,)
Index 26: X shape = (12, 2), y shape = (12,)
Index 27: X shape = (12, 2), y shape = (12,)
Index 28: X shape = (12, 

<ipython-input-28-7581b272b43a>:156: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results_subset = grouped.apply(lambda x: rolling_regression_tensorflow(x, window=12))  # Use reduced window
<ipython-input-28-7581b272b43a>:205: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_portfolios_subset = portfolio_data_subset.groupby('date').apply(


In [29]:
results_subset

,PERMNO,date,beta_mkt,beta_allocated
873,10014,1974-05-31,[0.0099375835],[0.0006890716]
874,10014,1974-07-31,[0.009630578],[0.003524893]
875,10014,1974-09-30,[0.005686051],[0.007750895]
876,10014,1974-10-31,[0.0088776965],[0.0063778646]
877,10014,1974-12-31,[0.012973026],[0.005700763]
...,...,...,...,...
738942,55300,1975-04-30,[0.023577109],[-0.0013738021]
738943,55300,1975-06-30,[0.025471896],[-0.0036365977]
738944,55300,1975-07-31,[0.026742974],[-0.006696314]
738945,55300,1975-09-30,[0.021883931],[-0.000120988116]


In [45]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Step 1: Load Stock Data
stock_data_path = "data.csv"  # File uploaded to Colab
stock_data = pd.read_csv(stock_data_path)

# Convert date column to datetime format
stock_data['date'] = pd.to_datetime(stock_data['date'])

# Filter data: Prices > $1, Exchange codes 1, 2, or 3, Share codes 10 or 11
filtered_data = stock_data[
    (stock_data['PRC'].abs() > 1) &
    (stock_data['EXCHCD'].isin([1, 2, 3])) &
    (stock_data['SHRCD'].isin([10, 11]))
]

# Calculate market equity (ME)
filtered_data['ME'] = filtered_data['PRC'].abs() * filtered_data['SHROUT']

# Calculate natural log of ME
filtered_data['ln_ME'] = np.log(filtered_data['ME'])

# Sort by PERMNO and date
filtered_data = filtered_data.sort_values(by=['PERMNO', 'date'])

# Step 2: Load Fama-French Factors
ff_factors_path = "F-F_Research_Data_Factors.csv"  # File uploaded to Colab

# Skip the first 3 rows (descriptive text) and use row 4 as the header
ff_factors = pd.read_csv(ff_factors_path, skiprows=3)

# Assign a label to the first column (date column)
ff_factors.columns = ['date'] + list(ff_factors.columns[1:])

# Drop rows where the 'date' column is NaN
ff_factors = ff_factors.dropna(subset=['date'])

# Filter out rows where the 'date' column does not match the YYYYMM format
ff_factors = ff_factors[ff_factors['date'].str.isdigit() & (ff_factors['date'].str.len() == 6)]

# Convert date column to datetime format
ff_factors['date'] = pd.to_datetime(ff_factors['date'], format='%Y%m')

# Rename columns for consistency
ff_factors.rename(columns={'Mkt-RF': 'mkt_excess_return'}, inplace=True)

# Ensure the dates align with the stock data
ff_factors = ff_factors.set_index('date').resample('ME').last().reset_index()  # Updated from 'M' to 'ME'

# Step 3: Load q-Factor Data (Replace with your assigned factor)
q_factors_path = "q5_factors_monthly_2023.csv"  # File uploaded to Colab
q_factors = pd.read_csv(q_factors_path)

# Combine year and month into a single date column
q_factors['date'] = pd.to_datetime(q_factors['year'].astype(str) + '-' + q_factors['month'].astype(str) + '-01')

# Rename columns for consistency (Replace 'R_ME' with your assigned factor)
q_factors.rename(columns={'R_MKT': 'mkt_excess_return', 'R_ME': 'allocated_factor'}, inplace=True)

# Drop unnecessary columns (year and month), but keep 'date', 'mkt_excess_return', and 'allocated_factor'
q_factors = q_factors[['date', 'mkt_excess_return', 'allocated_factor']]

# Ensure the dates align with the stock data
q_factors = q_factors.set_index('date').resample('ME').last().reset_index()  # Updated from 'M' to 'ME'
# Step 4: Merge Stock Data with Factors
merged_data = pd.merge(filtered_data, ff_factors, on='date', how='inner')
merged_data = pd.merge(merged_data, q_factors, on='date', how='inner')

# Resolve duplicate columns if necessary
if 'mkt_excess_return_x' in merged_data.columns and 'mkt_excess_return_y' in merged_data.columns:
    merged_data['mkt_excess_return'] = merged_data['mkt_excess_return_x']
    merged_data.drop(columns=['mkt_excess_return_x', 'mkt_excess_return_y'], inplace=True)

# Debug: Print columns after merging
#print("Merged Data Columns:", merged_data.columns)

# Ensure RET and RF columns are numeric
merged_data['RET'] = pd.to_numeric(merged_data['RET'], errors='coerce')
merged_data['RF'] = pd.to_numeric(merged_data['RF'], errors='coerce')

# Calculate excess returns for each stock
merged_data['excess_return'] = merged_data['RET'] - merged_data['RF']

# Step 5: Create a Smaller Subset for Testing
# Limit the date range to a shorter period (e.g., 1973–1983)
subset_data = merged_data[(merged_data['date'] >= '1973-01-01') & (merged_data['date'] <= '1983-12-31')]

# Randomly select a subset of PERMNOs (e.g., 100 stocks)
unique_permnos = subset_data['PERMNO'].unique()
sample_permnos = np.random.choice(unique_permnos, size=100, replace=False)  # Randomly select 100 PERMNOs
subset_data = subset_data[subset_data['PERMNO'].isin(sample_permnos)]

# Verify the subset
#print(f"Subset Data Shape: {subset_data.shape}")
#print(f"Unique PERMNOs in Subset: {len(subset_data['PERMNO'].unique())}")

# Step 6: Rolling Regression Function (Using TensorFlow for GPU Acceleration)
def rolling_regression_tensorflow(data, window=60):
    betas_mkt = []
    betas_allocated = []

    for i in range(window, len(data)):
        # Extract the rolling window
        window_data = data.iloc[i-window:i]

        # Drop rows with NaN values in any of the required columns
        window_data = window_data.dropna(subset=['mkt_excess_return', 'allocated_factor', 'excess_return'])

        # Skip if there are fewer than 2 observations (minimum required for regression)
        if len(window_data) < 2:
            betas_mkt.append(np.nan)
            betas_allocated.append(np.nan)
            continue

        # Define independent variables (MKT and Allocated Factor) and dependent variable (excess return)
        X = window_data[['mkt_excess_return', 'allocated_factor']].values
        y = window_data['excess_return'].values

        try:
            # Convert to TensorFlow tensors and move to GPU
            X_tf = tf.convert_to_tensor(X, dtype=tf.float32)
            y_tf = tf.convert_to_tensor(y, dtype=tf.float32)

            # Reshape y_tf to be a 2D array with shape (n_samples, 1)
            y_tf = tf.reshape(y_tf, (-1, 1))

            # Add a column of ones for the intercept term
            X_tf = tf.concat([tf.ones((X_tf.shape[0], 1), dtype=tf.float32), X_tf], axis=1)

            # Compute coefficients using the normal equation: β = (X^T X)^(-1) X^T y
            beta = tf.linalg.inv(tf.transpose(X_tf) @ X_tf) @ tf.transpose(X_tf) @ y_tf

            # Save the betas (skip the intercept term)
            betas_mkt.append(float(beta[1].numpy().item()))  # Ensure scalar value for MKT
            betas_allocated.append(float(beta[2].numpy().item()))  # Ensure scalar value for Allocated Factor
        except Exception as e:
            print(f"Error in regression at index {i}: {e}")
            betas_mkt.append(np.nan)
            betas_allocated.append(np.nan)

    # Return betas starting from the 61st month (window + 1)
    return pd.DataFrame({
        'date': data['date'][window:],
        'beta_mkt': betas_mkt,
        'beta_allocated': betas_allocated
    })

# Step 7: Apply Rolling Regression to Each Stock in the Subset
grouped = subset_data.groupby('PERMNO')
results_subset = grouped.apply(lambda x: rolling_regression_tensorflow(x, window=60))  # Use full window size

# Reset index to flatten the results
results_subset = results_subset.reset_index(level=0).rename(columns={'level_0': 'PERMNO'})

# Inspect the results
#print("Subset Results Shape:", results_subset.shape)
print(results_subset.head())



       PERMNO       date  beta_mkt  beta_allocated
14815   10233 1980-01-31  0.013674        0.004975
14816   10233 1980-02-29  0.012214        0.005926
14817   10233 1980-03-31  0.012368        0.007225
14818   10233 1980-04-30  0.014983        0.009569
14819   10233 1980-06-30  0.012705        0.012193


<ipython-input-45-82840bd6bc18>:152: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results_subset = grouped.apply(lambda x: rolling_regression_tensorflow(x, window=60))  # Use full window size


In [46]:
# Step 8: Portfolio Formation (Monthly Deciles)
def form_portfolios(data, beta_column, weight_column=None):
    """
    Forms 10 decile portfolios based on beta values.
    If weight_column is None, portfolios are equally-weighted.
    Otherwise, portfolios are value-weighted using the weight_column.
    """
    # Ensure beta_column contains valid scalar values
    data[beta_column] = pd.to_numeric(data[beta_column], errors='coerce')

    # Check if there are enough unique beta values for decile ranking
    unique_betas = data[beta_column].dropna().unique()
    if len(unique_betas) < 10:
        print(f"Not enough unique beta values ({len(unique_betas)}) to form 10 deciles.")
        return pd.Series({i: np.nan for i in range(10)})

    # Rank stocks into deciles based on beta values
    try:
        data['decile'] = pd.qcut(data[beta_column], q=10, labels=False)
    except ValueError as e:
        print(f"Error in pd.qcut: {e}")
        return pd.Series({i: np.nan for i in range(10)})

    # Group by decile and calculate portfolio returns
    if weight_column:
        # Value-weighted returns
        portfolio_returns = data.groupby('decile', group_keys=False)[['RET', weight_column]].apply(
            lambda x: np.average(x['RET'], weights=x[weight_column])
        )
    else:
        # Equally-weighted returns
        portfolio_returns = data.groupby('decile', group_keys=False)['RET'].mean()

    return portfolio_returns



In [47]:
# Step 9: Monthly Portfolio Returns
# Merge betas with stock data
portfolio_data_subset = pd.merge(results_subset, subset_data, on=['PERMNO', 'date'], how='inner')

# Form portfolios monthly
monthly_portfolios_subset = portfolio_data_subset.groupby('date').apply(
    lambda x: form_portfolios(x, beta_column='beta_allocated', weight_column='ME')
)

# Inspect the portfolio returns
print("Monthly Portfolios Subset:")
print(monthly_portfolios_subset.head())

Monthly Portfolios Subset:
decile             0         1         2         3         4         5  \
date                                                                     
1980-01-31 -0.068415  0.098075 -0.029916  0.020710  0.004054  0.010131   
1980-02-29 -0.069867  0.043998  0.025494 -0.147675 -0.049214 -0.019161   
1980-03-31 -0.086701 -0.119699 -0.037576 -0.139081 -0.147719 -0.039127   
1980-04-30  0.004274  0.049719  0.138137 -0.012741  0.133475 -0.077377   
1980-06-30  0.032764  0.303261  0.025694 -0.005209  0.026743  0.018310   

decile             6         7         8         9  
date                                                
1980-01-31  0.005976  0.082567  0.120182 -0.032857  
1980-02-29 -0.120109 -0.032027 -0.044222 -0.084781  
1980-03-31 -0.121361 -0.193025 -0.153272 -0.119625  
1980-04-30  0.056225  0.024429  0.178146  0.167353  
1980-06-30  0.045412  0.031112  0.065664  0.110193  


<ipython-input-47-d562f4d05c29>:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_portfolios_subset = portfolio_data_subset.groupby('date').apply(


In [48]:
# Step 10: Arbitrage Portfolio (Long Portfolio 10, Short Portfolio 1)
arbitrage_portfolio_subset = monthly_portfolios_subset.get(10, pd.Series(index=monthly_portfolios_subset.index, data=np.nan)) - \
                              monthly_portfolios_subset.get(1, pd.Series(index=monthly_portfolios_subset.index, data=np.nan))

In [49]:
arbitrage_portfolio_subset

,0
date,
1980-01-31,NaN
1980-02-29,NaN
1980-03-31,NaN
1980-04-30,NaN
1980-06-30,NaN
1980-07-31,NaN
1980-09-30,NaN
1980-10-31,NaN
1980-12-31,NaN


In [ ]:




# Save arbitrage portfolio returns to a CSV file
arbitrage_portfolio_path = "arbitrage_portfolio_returns_subset.csv"
arbitrage_portfolio_subset.to_csv(arbitrage_portfolio_path)

# Step 11: Performance Metrics
def calculate_metrics(returns):
    avg_return = returns.mean()
    std_return = returns.std()
    sharpe_ratio = avg_return / std_return
    t_stat = avg_return / (std_return / np.sqrt(len(returns)))
    return pd.Series({
        'Average Return': avg_return,
        'Standard Deviation': std_return,
        'Sharpe Ratio': sharpe_ratio,
        't-statistic': t_stat
    })

# Calculate metrics for arbitrage portfolio
if arbitrage_portfolio_subset.notna().sum() > 0:
    metrics_subset = calculate_metrics(arbitrage_portfolio_subset)
    print("Subset Metrics:")
    print(metrics_subset)
else:
    print("Arbitrage portfolio contains no valid returns.")
    metrics_subset = pd.Series({
        'Average Return': np.nan,
        'Standard Deviation': np.nan,
        'Sharpe Ratio': np.nan,
        't-statistic': np.nan
    })

# Save metrics to a CSV file
metrics_path = "arbitrage_portfolio_metrics_subset.csv"
metrics_subset.to_csv(metrics_path)